In [1]:
import numpy as np
import pickle

from sklearn.preprocessing import StandardScaler

# Preprocessing the data

In [2]:
raw_csv_data=np.loadtxt('C:/Users/armaod/Documents/Customer-Analysis/data/Audiobooks_data.csv', delimiter=',')

In [3]:
unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all =raw_csv_data[:,-1]

In [4]:
targets_all.sum()/len(targets_all)

0.1588327179778472

In [5]:
print(len(unscaled_inputs_all))
print(len(targets_all))

14084
14084


### Balancing tha dataset

In [6]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter+=1
        if zero_targets_counter> num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete (targets_all, indices_to_remove, axis=0)

In [7]:
print(len(unscaled_inputs_equal_priors))
print(len(targets_equal_priors))

4474
4474


### Standardize the input

In [8]:
scaler_deep_learning = StandardScaler()
scaled_inputs = scaler_deep_learning.fit_transform(unscaled_inputs_equal_priors)

In [10]:
pickle.dump(scaler_deep_learning, open('scaler_deep_learning.pkl', 'wb'))

### Shuffle the data

Dato il batching è meglio mescolare a caso il dataset

In [19]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [25]:
print(len(shuffled_inputs))
print(len(shuffled_targets))

4474
4474


### Split the dataset into train, validation, and test

In [30]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets)/test_samples_count)


1801.0 3579 0.5032131880413523
229.0 447 0.5123042505592841
207.0 448 0.46205357142857145


Notare come le proporzioni di tutti i set sono quasi bilanciate a 50-50

### Saving the datasets in *.npz

In [31]:
np.savez('audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('audiobooks_data_test', inputs=test_inputs, targets=test_targets)